In [1]:
#RAVDESS: Ryerson Audio-Visual Database of Emotional Speech and Song
#Data Preprocessing - Importing the Libraries

import librosa    #Python Library for analysing audio
import soundfile
import os, glob, pickle
import numpy as np

In [2]:
#Extract features (mfcc, chroma, mel) from a sound file

def extract_features(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype = "float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
            result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y = X, sr = sample_rate, n_mfcc = 40).T, axis = 0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis = 0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(X, sr = sample_rate).T, axis = 0)
            result = np.hstack((result, mel))
    return result

In [3]:
#Emotions in the RAVDESS dataset

emotions = {
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

In [4]:
#Emotions to observe

observed_emotions = ['calm', 'happy', 'fearful', 'disgust']

In [5]:
#importing the data and extracting features for each sound file

from sklearn.model_selection import train_test_split

def load_data(test_size=0.2):
    X, y = [], []
    for file in glob.glob("ravdessdata/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_features(file, mfcc = True, chroma = True, mel = True)
        X.append(feature)
        y.append(emotion)
    return train_test_split(np.array(X), y, test_size = test_size, random_state = 9)

In [6]:
#Splitting the dataset

X_train, X_test, y_train, y_test = load_data(test_size = 0.25)

In [7]:
#Getting the shape of the training and testing datasets

print((X_train.shape[0], X_test.shape[0]))

(576, 192)


In [8]:
#Getting the number of features extracted

print(f'Features extracted: {X_train.shape[1]}')

Features extracted: 180


In [9]:
#Initializing the Multi Layer Perceptron Classifier

from sklearn.neural_network import MLPClassifier
classifier = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (300,), 
                    learning_rate='adaptive', max_iter = 500)

In [10]:
#Training the model

classifier.fit(X_train,y_train)

MLPClassifier(activation='relu', alpha=0.01, batch_size=256, beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(300,), learning_rate='adaptive',
              learning_rate_init=0.001, max_iter=500, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=None, shuffle=True, solver='adam', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [11]:
#Prediction for the test set

y_pred = classifier.predict(X_test)

In [12]:
#Calculating the accuracy of our model

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true = y_test, y_pred = y_pred)

In [13]:
#Printing the accuracy

print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 72.40%
